<a href="https://colab.research.google.com/github/vvvu/potential-chainsaw/blob/main/pytorch-tutorial/%5BBasics%5D_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [12]:
# MNIST Hyper Parameters
input_size = 28 * 28 # 784 pixels
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True)

test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.ToTensor())

# Data Loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

# Logistic Regression Model => Classification
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Reshape images to (batch_size, input_size)
    images = images.reshape(-1, input_size) # [100, 784]

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))  

Epoch [1/5], Step [100/600], Loss: 2.2377
Epoch [1/5], Step [200/600], Loss: 2.1369
Epoch [1/5], Step [300/600], Loss: 2.0481
Epoch [1/5], Step [400/600], Loss: 1.9134
Epoch [1/5], Step [500/600], Loss: 1.8844
Epoch [1/5], Step [600/600], Loss: 1.7642
Epoch [2/5], Step [100/600], Loss: 1.7115
Epoch [2/5], Step [200/600], Loss: 1.6392
Epoch [2/5], Step [300/600], Loss: 1.6185
Epoch [2/5], Step [400/600], Loss: 1.5103
Epoch [2/5], Step [500/600], Loss: 1.4989
Epoch [2/5], Step [600/600], Loss: 1.4901
Epoch [3/5], Step [100/600], Loss: 1.4119
Epoch [3/5], Step [200/600], Loss: 1.3086
Epoch [3/5], Step [300/600], Loss: 1.3244
Epoch [3/5], Step [400/600], Loss: 1.3710
Epoch [3/5], Step [500/600], Loss: 1.2870
Epoch [3/5], Step [600/600], Loss: 1.2156
Epoch [4/5], Step [100/600], Loss: 1.1655
Epoch [4/5], Step [200/600], Loss: 1.0116
Epoch [4/5], Step [300/600], Loss: 1.2962
Epoch [4/5], Step [400/600], Loss: 1.0903
Epoch [4/5], Step [500/600], Loss: 1.1003
Epoch [4/5], Step [600/600], Loss:

In [13]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
'''
1. 实际上，在Test阶段我们无需计算梯度，我们只需要对比对应的label是否相同
2. 但是PyTorch默认会让「所有的操作都进行Computational Graph的生成」，对于Tensor的计算操作
  默认是要进行Computational Graph的构建的
3. 在这种情况下，可以使用`with torch.no_grad():`，强制之后的内容不进行Compuational Graph的构建
4. 从而节省内存资源
'''

with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, input_size)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    '''
    _ : 代表具体的value，即分类到某一特定类的概率
    predicted : 代表value所在的index

    注意我们这里是一个分类问题，实际上我们并不关注具体的value是多少，我们关注的是某个数据
    被分到了哪一类，即index

    这里的数字1实际上代表`dim = 1`，表示输出所在行的最大值。如果`dim = 0`，则代表输出
    所在列的最大值。

    在MNIST问题中：网络预测结果中，行代表样本，列代表类别，所以这里应该用`dim = 1`，因为我们
    需要输出每个样本的预测类别
    '''
    total += labels.size(0)
    '''
    这里的labels.size(0)实际上就是我们之前设置的test_loader的batch_size = 100
    test_loader每次读入batch_size的数字，我们需要计算total，则累加即可
    Tips: 虽然没有验证，但如果剩余数据数量不足batch_size所要求数量，这里的labels.size(0)
    应该会返回当前batch「实际读入的数量」而非batch_size。
    '''
    correct += (predicted == labels).sum()

  print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
  
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 83.51000213623047 %
